In [1]:
# !pip install --user pandas-profiling

In [2]:
import os

import numpy as np
import pandas as pd
import pandas_profiling
import datetime

In [3]:
!mkdir -p data

In [161]:
RAW_DATA_PATH = './raw_data/'
DATA_PATH = './data'

In [5]:
!ls raw_data/

Blitz		  test_users.csv  train_clicks.csv  train_shares.csv
descriptions.csv  themes.csv	  train_likes.csv   user_information.csv


In [144]:
raw_data_likes = pd.read_csv(os.path.join(RAW_DATA_PATH, 'train_likes.csv'))
raw_data_likes['day'] = pd.to_datetime(raw_data_likes['day'])
raw_data_likes.head()

,user_id,picture_id,day
0,490982,298754,2019-03-21
1,490989,1060978,2019-02-17
2,490989,976274,2019-02-17
3,490989,976273,2019-02-17
4,490989,976272,2019-02-17


In [153]:
def get_user_day_ordinal(df):
    return np.concatenate(
        (
            df
                .groupby(['user_id', 'day'] , sort=False)['day']
                .apply(lambda s: np.arange(len(s)))
        ).tolist()
    )

In [152]:
def get_pseudo_ts(df):
    return np.concatenate(
        (
            df
                .groupby(['user_id', 'day'], sort=False)['day']
                .apply(lambda s: s.min() +  ((datetime.timedelta(days=1) / len(s)) * np.arange(len(s))))
        ).tolist()
    )

In [ ]:
# def filter_users_pictures(df):
#     df_user_actions_count = df.groupby('user_id')['picture_id'].count()
#     selected_users = df_user_action_count[df_user_actions_count > 1].index.tolist()
#     df = df[df['user_id'].isin(selected_users)]
    
#     df_user_actions_count = cutted_train_clicks.groupby('user_id')['picture_id'].count()
#     selected_users = df_user_action_count[df_user_actions_count > 1].index.tolist()
#     df = df[df['user_id'].isin(selected_users)]
    
#     df_picture_actions_count = df.groupby('picture_id')['user_id'].count()
#     selected_pictures = df_picture_actions_count[df_picture_actions_count > 1].index.tolist()
#     df = df[df['picture_id'].isin(selected_pictures)]
    
#     return df

##### Likes

In [154]:
raw_data_likes['user_day_ordinal'] = get_user_day_ordinal(raw_data_likes)
raw_data_likes['pseudo_ts'] = get_pseudo_ts(raw_data_likes)
raw_data_likes.sort_values(['user_id', 'day']).head(10)

,user_id,picture_id,day,user_day_ordinal,pseudo_ts
2515,0,361219,2019-03-05,0,2019-03-05 00:00:00
2516,0,179639,2019-03-05,1,2019-03-05 08:00:00
2517,0,222809,2019-03-05,2,2019-03-05 16:00:00
2514,7,1141076,2019-02-13,0,2019-02-13 00:00:00
2505,12,134491,2019-02-09,0,2019-02-09 00:00:00
2506,13,691209,2019-02-14,0,2019-02-14 00:00:00
2507,13,384307,2019-02-14,1,2019-02-14 12:00:00
2508,14,571167,2019-03-17,0,2019-03-17 00:00:00
2509,15,1173294,2019-02-02,0,2019-02-02 00:00:00
2510,19,662204,2019-03-07,0,2019-03-07 00:00:00


In [187]:
raw_data_likes.to_hdf(os.path.join(DATA_PATH, 'step0_likes.h5'), index=None, key='step0')

In [175]:
all(new_raw_data_likes == raw_data_likes)

True

##### Clicks

In [151]:
raw_data_clicks = pd.read_csv(os.path.join(RAW_DATA_PATH, 'train_clicks.csv'))
raw_data_clicks['day'] = pd.to_datetime(raw_data_clicks['day'])
raw_data_clicks.head()

,user_id,picture_id,day
0,1442,546149,2019-02-09
1,1442,546149,2019-02-09
2,1442,1242875,2019-02-16
3,1442,1242875,2019-02-16
4,1442,1242891,2019-02-16


In [178]:
raw_data_clicks['user_day_ordinal'] = get_user_day_ordinal(raw_data_clicks)
raw_data_clicks['pseudo_ts'] = get_pseudo_ts(raw_data_clicks)
raw_data_clicks.sort_values(['user_id', 'day'], ascending=False).tail(10)

,user_id,picture_id,day,user_day_ordinal,pseudo_ts
42076,0,222809,2019-03-05,0,2019-03-05 00:00:00
42077,0,204858,2019-03-05,1,2019-03-05 04:00:00
42078,0,216014,2019-03-05,2,2019-03-05 08:00:00
42079,0,323106,2019-03-05,3,2019-03-05 12:00:00
42080,0,177075,2019-03-05,4,2019-03-05 16:00:00
42081,0,958096,2019-03-05,5,2019-03-05 20:00:00
42072,0,50605,2019-02-07,0,2019-02-07 00:00:00
42073,0,1194764,2019-02-07,1,2019-02-07 06:00:00
42074,0,210328,2019-02-07,2,2019-02-07 12:00:00
42075,0,280717,2019-02-07,3,2019-02-07 18:00:00


In [179]:
raw_data_clicks.to_hdf(os.path.join(DATA_PATH, 'step0_clicks.h5'), index=None, key='step0')

##### Shares

In [157]:
raw_data_shares = pd.read_csv(os.path.join(RAW_DATA_PATH, 'train_shares.csv'))
raw_data_shares['day'] = pd.to_datetime(raw_data_shares['day'])
raw_data_shares.head()

,user_id,picture_id,day
0,24986,216354,2019-02-15
1,25046,514369,2019-03-08
2,25058,1066857,2019-02-05
3,25058,481307,2019-02-05
4,25058,636884,2019-02-05


In [158]:
raw_data_shares['user_day_ordinal'] = get_user_day_ordinal(raw_data_shares)
raw_data_shares['pseudo_ts'] = get_pseudo_ts(raw_data_shares)
raw_data_shares.sort_values(['user_id', 'day']).tail(10)

,user_id,picture_id,day,user_day_ordinal,pseudo_ts
47609,317713,467313,2019-03-21,1,2019-03-21 12:00:00
47605,337088,1380260,2019-03-03,0,2019-03-03 00:00:00
47610,342704,1482510,2019-03-20,0,2019-03-20 00:00:00
15680,430530,1353981,2019-02-26,0,2019-02-26 00:00:00
15681,430530,1356754,2019-02-26,1,2019-02-26 12:00:00
47611,456535,1451478,2019-03-08,0,2019-03-08 00:00:00
47612,482539,1431927,2019-03-14,0,2019-03-14 00:00:00
47613,482539,1431927,2019-03-14,1,2019-03-14 06:00:00
47614,482539,1431927,2019-03-14,2,2019-03-14 12:00:00
47615,482539,1431927,2019-03-14,3,2019-03-14 18:00:00


In [180]:
raw_data_shares.to_hdf(os.path.join(DATA_PATH, 'step0_shares.h5'), index=None, key='step0')

##### test_users

In [159]:
test_users = pd.read_csv(os.path.join(RAW_DATA_PATH, 'test_users.csv'))
test_users.head()

,user_id
0,103
1,178
2,1069
3,1310
4,2108


# Mapping

##### User

In [199]:
# user mapping
all_user_ids = np.sort(np.unique(
    raw_data_likes['user_id'].unique().tolist() +
    raw_data_clicks['user_id'].unique().tolist() +
    raw_data_shares['user_id'].unique().tolist() +
    test_users['user_id'].unique().tolist()
))

user_mapping = pd.DataFrame({'old': all_user_ids, 'new': np.arange(len(all_user_ids))})

In [201]:
user_mapping.to_csv(os.path.join(DATA_PATH, 'step0_user_mapping.csv'), index=False)

##### Pic

In [198]:
# picture mapping
all_picture_ids = np.sort(np.unique(
    raw_data_likes['picture_id'].unique().tolist() +
    raw_data_clicks['picture_id'].unique().tolist() +
    raw_data_shares['picture_id'].unique().tolist() +
    raw_picture_descriptions['picture_id'].unique().tolist()
))

picture_mapping = pd.DataFrame({'old': all_picture_ids, 'new': np.arange(len(all_picture_ids))})

In [205]:
len(set(raw_picture_descriptions['picture_id'].unique().tolist()) & set(picture_mapping['old'].unique().tolist()))

1379875

In [206]:
picture_mapping.to_csv(os.path.join(DATA_PATH, 'step0_picture_mapping.csv'), index=False)

# Descriptions

In [14]:
raw_picture_descriptions = pd.read_csv(os.path.join(RAW_DATA_PATH, 'descriptions.csv'))
raw_picture_descriptions.head()

,picture_id,description
0,1171496,734475 487260 884706 5941 48255 147285 969800
1,1171497,488295 261184 466630 235887 642445 465419 9297...
2,1171498,724435 1003828 164020
3,1171499,274351 96637 259690 590856 761704 148158 16580...
4,1171500,235887 117104 952749 285126 288206


In [15]:
# attribute mapping
pictures = []
descs = []
for picture_id, picture_descs in raw_picture_descriptions[['picture_id', 'description']].values:
    pictures += np.repeat([picture_id], repeats=len(picture_descs)).tolist()
    descs += picture_descs

In [16]:
picture_descs_df = pd.DataFrame({'picture_id': pictures, 'description_uid': descs})

grouper_attr = picture_descs_df.groupby('description_uid').grouper
desc_mapping = pd.DataFrame({'new': grouper_attr.group_info[1], 'old': grouper_attr.levels[0]})

In [182]:
desc_mapping.to_csv(os.path.join(DATA_PATH, 'desc_mapping.csv'), index=False)

In [188]:
!ls -alh data/

total 145M
drwxrwxrwx 1 root root  424 Jun  1 11:28 .
drwxrwxrwx 1 root root 4.0K Jun  1 11:28 ..
-rwxrwxrwx 1 root root   53 Jun  1 11:12 desc_mapping.csv
-rwxrwxrwx 1 root root 109M Jun  1 11:11 step0_clicks.h5
-rwxrwxrwx 1 root root  15M Jun  1 11:28 step0_likes.h5
-rwxrwxrwx 1 root root 2.3M Jun  1 11:11 step0_shares.h5
-rwxrwxrwx 1 root root  15M Jun  1 11:09 step0_train_likes.h5
-rwxrwxrwx 1 root root 5.8M Jun  1 11:12 user_mapping.csv
